In [2]:
# Install dependencies (run once)
!pip install -q gradio sentence-transformers faiss-cpu python-docx PyMuPDF beautifulsoup4 requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install -q langchain langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [5]:
import os
import tempfile
import gradio as gr
from langchain.schema import HumanMessage
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

In [6]:
# ==== CONFIG ====
EMBED_MODEL = "all-MiniLM-L6-v2"

In [7]:
# ==== INIT ====
embeddings = HuggingFaceEmbeddings(model_name=EMBED_MODEL)
vectorstore = None

/tmp/ipython-input-4057968039.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=EMBED_MODEL)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
# ---------- Utils: file -> text ----------
def extract_text_from_pdf(path_or_bytes):
    text = ""
    doc = fitz.open(path_or_bytes if isinstance(path_or_bytes, (str, Path)) else None, stream=path_or_bytes.read() if hasattr(path_or_bytes,'read') else None)
    for page in doc:
        page_text = page.get_text("text")
        if page_text:
            text += page_text + "\n"
    return text

def extract_text_from_docx(path_or_bytes):
    if hasattr(path_or_bytes, "read"):
        # save to temp then read
        tmp = "tmp_docx.docx"
        with open(tmp, "wb") as f:
            f.write(path_or_bytes.read())
        path = tmp
    else:
        path = path_or_bytes
    doc = docx.Document(path)
    text = "\n".join([p.text for p in doc.paragraphs])
    return text

def extract_text_from_txt(path_or_bytes):
    if hasattr(path_or_bytes, "read"):
        return path_or_bytes.read().decode("utf-8")
    with open(path_or_bytes, "r", encoding="utf-8") as f:
        return f.read()

def extract_text_from_html(path_or_bytes):
    raw = extract_text_from_txt(path_or_bytes) if not hasattr(path_or_bytes,"read") else path_or_bytes.read().decode("utf-8")
    return BeautifulSoup(raw, "html.parser").get_text(separator="\n")

def load_file_to_text(uploaded_file):
    name = uploaded_file.name
    ext = name.split(".")[-1].lower()
    if ext == "pdf":
        return extract_text_from_pdf(uploaded_file)
    if ext in ("docx","doc"):
        return extract_text_from_docx(uploaded_file)
    if ext in ("txt","md"):
        return extract_text_from_txt(uploaded_file)
    if ext == "html":
        return extract_text_from_html(uploaded_file)
    return ""


# ---------- Chunking ----------
def chunk_text(text: str, chunk_size_words: int = 400, overlap: int = 80) -> List[str]:
    words = text.split()
    if not words:
        return []
    chunks = []
    i = 0
    while i < len(words):
        chunk = words[i:i+chunk_size_words]
        chunks.append(" ".join(chunk))
        i += (chunk_size_words - overlap)
    return chunks

In [11]:
def build_faiss_index(chunks: List[str]):
    embeddings = embedder.encode(chunks, convert_to_numpy=True, show_progress_bar=False)
    faiss.normalize_L2(embeddings)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)
    return index, embeddings

def retrieve_top_k(index, query: str, chunks: List[str], k: int = 5):
    q_emb = embedder.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(q_emb)
    D, I = index.search(q_emb, k)
    results = []
    for score, idx in zip(D[0], I[0]):
        results.append({"text": chunks[idx], "score": float(score), "idx": int(idx)})
    return results

# ---------- Groq API (chat completions) ----------
def call_groq_chat(api_key: str, model: str, messages: List[dict], max_tokens: int = 256, temperature: float = 0.0):
    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    body = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
        "max_tokens": max_tokens,
    }
    resp = requests.post(url, headers=headers, json=body, timeout=60)
    if resp.status_code != 200:
        raise RuntimeError(f"Groq API error {resp.status_code}: {resp.text}")
    data = resp.json()
    # follow OpenAI ChatCompletions response shape assumption
    content = data.get("choices", [{}])[0].get("message", {}).get("content")
    return content

In [12]:
# ---------- Imports ----------
import os, io, json, textwrap
from typing import List
from pathlib import Path
import requests
import gradio as gr
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import fitz  # PyMuPDF
import docx
from bs4 import BeautifulSoup

In [13]:
# ---------- Embeddings & FAISS ----------
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
embedder = SentenceTransformer(EMBED_MODEL)  # will use CPU/GPU automatically

In [14]:
# ---------- Prompt builders ----------
MODEL_NAME = "meta-llama/llama-4-scout-17b-16e-instruct"

In [15]:
def make_contexted_prompt_for_qa(query: str, hits: List[dict]):
    ctx = "\n\n---\n\n".join([h["text"] for h in hits])
    prompt = textwrap.dedent(f"""You are a helpful assistant. Use ONLY the context below to answer the user question. If the answer is not contained in the context, say "Not enough information in the document."

Context:
{ctx}

Question:
{query}

Answer concisely:""")
    return prompt

def make_prompt_for_summary(hits: List[dict]):
    ctx = "\n\n---\n\n".join([h["text"] for h in hits])
    prompt = textwrap.dedent(f"""Summarize the content below in 6-10 concise sentences. Focus on the main ideas and facts.

Content:
{ctx}

Summary:""")
    return prompt

def make_prompt_for_keypoints(hits: List[dict], max_points=8):
    ctx = "\n\n---\n\n".join([h["text"] for h in hits])
    prompt = textwrap.dedent(f"""Extract up to {max_points} concise bullet points capturing the most important facts from the content below. Use short sentences and start each bullet with a dash.

Content:
{ctx}

Bullets:""")
    return prompt

def make_prompt_for_translate(hits: List[dict], target_language: str):
    ctx = "\n\n---\n\n".join([h["text"] for h in hits])
    prompt = textwrap.dedent(f"""Translate the important points from the content below into {target_language}. Keep the meaning intact and be concise.

Content:
{ctx}

Translation ({target_language}):""")
    return prompt


In [16]:
# ---------- High-level operations ----------
def process_upload_and_index(uploaded_file):
    text = load_file_to_text(uploaded_file)
    if not text.strip():
        return {"error": "No extractable text found in the file. If this is a scanned PDF, use OCR first."}
    chunks = chunk_text(text, chunk_size_words=400, overlap=80)
    index, embeddings = build_faiss_index(chunks)
    # Return objects needed later (we'll keep them as state in UI)
    return {"ok": True, "chunks": chunks, "index": index}

def do_qa(api_key, index, chunks, question):
    hits = retrieve_top_k(index, question, chunks, k=5)
    prompt = make_contexted_prompt_for_qa(question, hits)
    messages = [{"role":"user","content": prompt}]
    resp = call_groq_chat(api_key, MODEL_NAME, messages, max_tokens=400, temperature=0.0)
    return {"answer": resp, "evidence": hits}

def do_summary(api_key, index, chunks):
    # fetch broader context by using a generic query or concatenating top chunks
    # we'll rank by a simple heuristic: choose top chunks by embedding similarity to the whole doc (approx: query="summary")
    hits = retrieve_top_k(index, "summarize document", chunks, k=6)
    prompt = make_prompt_for_summary(hits)
    messages = [{"role":"user","content": prompt}]
    resp = call_groq_chat(api_key, MODEL_NAME, messages, max_tokens=400, temperature=0.0)
    return {"summary": resp, "evidence": hits}

def do_keypoints(api_key, index, chunks):
    hits = retrieve_top_k(index, "key points", chunks, k=6)
    prompt = make_prompt_for_keypoints(hits, max_points=8)
    messages = [{"role":"user","content": prompt}]
    resp = call_groq_chat(api_key, MODEL_NAME, messages, max_tokens=400, temperature=0.0)
    return {"key_points": resp, "evidence": hits}

def do_translate(api_key, index, chunks, target_language):
    hits = retrieve_top_k(index, "important content", chunks, k=6)
    prompt = make_prompt_for_translate(hits, target_language)
    messages = [{"role":"user","content": prompt}]
    resp = call_groq_chat(api_key, MODEL_NAME, messages, max_tokens=400, temperature=0.0)
    return {"translation": resp, "evidence": hits}

In [17]:
# ---------- Gradio UI with state ----------
with gr.Blocks(title="Universal Doc RAG (Groq + Embeddings)") as demo:
    gr.Markdown("## 📄 Document Q&A / Summarize / Key Points / Translate\nUpload a document, then click a button to run an operation. Enter your Groq API key (kept only in runtime memory).")
    with gr.Row():
        file_in = gr.File(label="Upload document (PDF, DOCX, TXT, HTML)")
        api_key_in = gr.Textbox(label="Groq API Key", type="password", placeholder="Paste your Groq API key here", elem_id="groq_key")
        load_btn = gr.Button("Process & Index File")
    status_out = gr.Textbox(label="Status", interactive=False)
    gr.Markdown("### Operations")
    with gr.Row():
        qa_btn = gr.Button("❓ Ask Question")
        summary_btn = gr.Button("📜 Summarize Document")
        keypoints_btn = gr.Button("📑 Extract Key Points")
        translate_btn = gr.Button("🌍 Translate Document")
    question_in = gr.Textbox(label="Question (for Q&A)", placeholder="Type your question here")
    lang_dropdown = gr.Dropdown(["English","Hindi","French","Spanish","German","Japanese","Arabic"], value="Hindi",label="Translate to")
    output_box = gr.Textbox(label="Output", lines=12)
    evidence_box = gr.Textbox(label="Evidence (top chunks)", lines=8)

    # state holders
    state_index = gr.State()
    state_chunks = gr.State()

    def on_load(file_obj, groq_key):
        if file_obj is None:
            return {"status":"No file uploaded."}, None, None
        try:
            res = process_upload_and_index(file_obj)
            if "error" in res:
                return {"status": res["error"]}, None, None
            return {"status": f"Indexed {len(res['chunks'])} chunks. Ready."}, res["index"], res["chunks"]
        except Exception as e:
            return {"status": f"Error processing file: {e}"}, None, None

    def on_qa(groq_key, index_obj, chunks_obj, question):
        if not groq_key:
            return "Error: Groq API key required.", ""
        if index_obj is None:
            return "Error: No indexed document. Please Process & Index first.", ""
        try:
            out = do_qa(groq_key, index_obj, chunks_obj, question)
            return out["answer"], json.dumps(out["evidence"], indent=2)
        except Exception as e:
            return f"API/Runtime error: {e}", ""

    def on_summary(groq_key, index_obj, chunks_obj):
        if not groq_key:
            return "Error: Groq API key required.", ""
        if index_obj is None:
            return "Error: No indexed document. Please Process & Index first.", ""
        try:
            out = do_summary(groq_key, index_obj, chunks_obj)
            return out["summary"], json.dumps(out["evidence"], indent=2)
        except Exception as e:
            return f"API/Runtime error: {e}", ""

    def on_keypoints(groq_key, index_obj, chunks_obj):
        if not groq_key:
            return "Error: Groq API key required.", ""
        if index_obj is None:
            return "Error: No indexed document. Please Process & Index first.", ""
        try:
            out = do_keypoints(groq_key, index_obj, chunks_obj)
            return out["key_points"], json.dumps(out["evidence"], indent=2)
        except Exception as e:
            return f"API/Runtime error: {e}", ""

    def on_translate(groq_key, index_obj, chunks_obj, language):
        if not groq_key:
            return "Error: Groq API key required.", ""
        if index_obj is None:
            return "Error: No indexed document. Please Process & Index first.", ""
        try:
            out = do_translate(groq_key, index_obj, chunks_obj, language)
            return out["translation"], json.dumps(out["evidence"], indent=2)
        except Exception as e:
            return f"API/Runtime error: {e}", ""

    load_btn.click(fn=on_load, inputs=[file_in, api_key_in], outputs=[status_out, state_index, state_chunks])
    qa_btn.click(fn=on_qa, inputs=[api_key_in, state_index, state_chunks, question_in], outputs=[output_box, evidence_box])
    summary_btn.click(fn=on_summary, inputs=[api_key_in, state_index, state_chunks], outputs=[output_box, evidence_box])
    keypoints_btn.click(fn=on_keypoints, inputs=[api_key_in, state_index, state_chunks], outputs=[output_box, evidence_box])
    translate_btn.click(fn=on_translate, inputs=[api_key_in, state_index, state_chunks, lang_dropdown], outputs=[output_box, evidence_box])

# Launch. In Colab use share=True to get a public link.
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ac60afdfe69d7cec30.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
